# Understand Events and their Effects

Effects are useful to create conditional relationships in your data: if some events happen `Effect`s can be configured to modify vectors in order to reflect the induced change of the event.

In [1]:
%pip install superlinked==3.21.0

In [2]:
import pandas as pd

from superlinked.framework.common.schema.event_schema import event_schema
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String
from superlinked.framework.common.schema.schema_reference import SchemaReference
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.index.effect import Effect
from superlinked.framework.dsl.query.param import Param

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

## Setting up event schemas

Events generally have:
- `SchemaReference`s: these contain ids that are resolved in the referenced schema. These reflect the items which were constituents of the event.
- event_type as a string: used to group events so that `Effect`s can be applied to a subset of events
- and id of course

In [3]:
@schema
class Paragraph:
    id: IdField
    body: String


@schema
class User:
    id: IdField
    interest: String


@event_schema
class Event:
    id: IdField
    paragraph: SchemaReference[Paragraph]
    user: SchemaReference[User]
    event_type: String


paragraph = Paragraph()
user = User()
event = Event()

relevance_space = TextSimilaritySpace(
    text=[user.interest, paragraph.body],
    model="sentence-transformers/all-mpnet-base-v2",
)
index = Index(
    spaces=relevance_space,
    effects=[
        Effect(
            relevance_space,
            event.user,
            0.8 * event.paragraph,
            event.event_type == "read",
        )
    ],
)

In [4]:
source_paragraph: InMemorySource = InMemorySource(paragraph)
source_user: InMemorySource = InMemorySource(user)
source_event: InMemorySource = InMemorySource(event)
executor = InMemoryExecutor(
    sources=[source_paragraph, source_user, source_event], indices=[index]
)
app = executor.run()

In [5]:
source_paragraph.put(
    [
        {"id": "paragraph-1", "body": "Glorious animals live in the wilderness."},
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
        },
    ]
)

source_user.put([{"id": "user-1", "interest": "I am interested in wild animals."}])

source_event.put(
    [
        {
            "id": "event-1",
            "paragraph": "paragraph-2",
            "user": "user-1",
            "event_type": "read",
        }
    ]
)

In [6]:
query = Query(index).find(paragraph).with_vector(user, Param("user_id"))

In [7]:
result = app.query(
    query,
    user_id="user-1",
)

## The power of events

Even though the user expressed interest in wild animals, as he interacted with the other document about AI the preferences shifted towards the actual experienced interests.

In [8]:
pd.DataFrame([entry.stored_object for entry in result.entries])

,body,id
0,Growing computation power enables advancements in AI.,paragraph-2
1,Glorious animals live in the wilderness.,paragraph-1
